## Pasos para la implementacion y uso del algoritmo Recocido Simulado para VRP

### Paso 1: Representacion de Datos con Pandas

El primer paso es cargar y organizar los datos para que el algoritmo pueda acceder a ellos de manera eficiente

Elementos a extraer:
- datos_distribucion_tiendas.xlsx (Nodos: 10 CD y 90 Tiendas) (Lista de Nodos/Clientes con sus atributos. Demanda y Capacidad).
- matriz_distancias.xlsx (Distancia entre cada par de nodos) (Matriz NxN de Distancias que serian sus costos).
- matriz_costos_combustible.xlsx (Costos de combustible entre cada par de nodos) (Matriz NxN de Costos de Combustible que serian sus costos).

In [3]:
import numpy as np
import pandas as pd
import random
import math

df_nodos = pd.read_excel("./Datos/datos_distribucion_tiendas.xlsx")
df_distancias = pd.read_excel("./Datos/matriz_distancias.xlsx")
df_costos = pd.read_excel("./Datos/matriz_costos_combustible.xlsx")

### 1.1 Identificación de Nodos

In [4]:
# 1.1. Identificación de Nodos
N_NODOS = len(df_nodos) # 10 CDs + 90 Tiendas = 100
CD_INDICES = list(range(10)) # Nodos 0 a 9
TIENDA_INDICES = list(range(10, N_NODOS)) # Nodos 10 a 99

# 1.2. Matriz de Costo Total y Matriz de Distancia
# Las matrices en CSV contienen una columna de índice y 100 columnas de datos.
columnas_datos = [f'Nodo_{i}' for i in range(1, N_NODOS + 1)]

# MATRIZ_COSTO_TOTAL: Costo es directo de la matriz de combustible
matriz_costos_combustible = df_costos[columnas_datos].values
MATRIZ_COSTO_TOTAL = matriz_costos_combustible

## Paso 2: Estructura de la Solución y Función de Objetivo

In [5]:

def calcular_costo_solucion(solucion):
    """
    Calcula el costo total de la solución sumando los costos de todos los segmentos de ruta.
    """
    costo_total = 0.0
    
    for ruta in solucion:
        if len(ruta) < 2:
            continue

        # Sumar costos de cada segmento (i, j) en la ruta
        for i in range(len(ruta) - 1):
            nodo_inicio = ruta[i]
            nodo_fin = ruta[i+1]
            costo_total += MATRIZ_COSTO_TOTAL[nodo_inicio, nodo_fin]
            
    return costo_total

## Paso 3: Componentes del Recocido Simulado

### 3.1 Generar solución inicial

In [6]:
def generar_solucion_inicial():
    """
    Genera una solución inicial aleatoria: asigna todas las tiendas a un CD aleatorio
    y luego secuencia las tiendas aleatoriamente dentro de cada ruta.
    """
    solucion = [[cd] for cd in CD_INDICES]  # Inicializa una ruta con cada CD
    tiendas_a_asignar = TIENDA_INDICES[:]
    random.shuffle(tiendas_a_asignar)
    
    # Asignación aleatoria de tiendas a CDs
    for tienda in tiendas_a_asignar:
        cd_idx = random.choice(CD_INDICES)
        solucion[cd_idx].append(tienda)
        
    # Completar las rutas, añadiendo el CD de regreso al final
    for i in range(len(solucion)):
        if len(solucion[i]) > 0:
            solucion[i].append(solucion[i][0])
            
    # Barajar el orden de las tiendas dentro de cada ruta (secuenciación TSP aleatoria)
    for ruta in solucion:
        if len(ruta) > 2:
            tiendas_ruta = ruta[1:-1]
            random.shuffle(tiendas_ruta)
            ruta[1:-1] = tiendas_ruta
            
    return solucion

### 3.2 Función de vecindad

In [7]:
def generar_vecina(solucion_actual):
    """
    Genera una solución vecina mediante un movimiento de vecindad aleatorio:
    1. Inter-ruta (cambio de CD)
    2. Intra-ruta (intercambio de tiendas en la misma ruta - Swap)
    """
    solucion = [ruta[:] for ruta in solucion_actual]  # Copia profunda
    movimiento = random.choice(['inter', 'intra'])
    
    if movimiento == 'inter':
        # Mover una tienda de una ruta (CD) a otra (CD)
        rutas_con_tiendas = [i for i, r in enumerate(solucion) if len(r) > 2]
        if not rutas_con_tiendas: return solucion
        
        idx_ruta_origen = random.choice(rutas_con_tiendas)
        ruta_origen = solucion[idx_ruta_origen]
        
        # Seleccionar una tienda para mover (índices 1 a len(ruta)-2)
        idx_tienda_en_ruta = random.randint(1, len(ruta_origen) - 2)
        tienda_movida = ruta_origen.pop(idx_tienda_en_ruta)
        
        idx_ruta_destino = random.choice(CD_INDICES)
        ruta_destino = solucion[idx_ruta_destino]
        
        # Insertar la tienda en la ruta de destino (en una posición aleatoria válida)
        if len(ruta_destino) < 2:
             ruta_destino.insert(1, tienda_movida)
        else:
             idx_insercion = random.randint(1, len(ruta_destino) - 1)
             ruta_destino.insert(idx_insercion, tienda_movida)
             
    elif movimiento == 'intra':
        # Intercambiar dos tiendas DENTRO de la misma ruta (Swap)
        rutas_elegibles = [i for i, r in enumerate(solucion) if len(r) >= 4]
        if not rutas_elegibles: return solucion

        idx_ruta = random.choice(rutas_elegibles)
        ruta = solucion[idx_ruta]

        idx1 = random.randint(1, len(ruta) - 2)
        idx2 = random.randint(1, len(ruta) - 2)
        
        while idx1 == idx2:
             idx2 = random.randint(1, len(ruta) - 2)

        ruta[idx1], ruta[idx2] = ruta[idx2], ruta[idx1]

    return solucion

### 3.3 Recocido Simulado

In [8]:


def recocido_simulado(T_inicial, T_final, alpha, max_iteraciones_T):
    """
    Implementación principal del algoritmo de Recocido Simulado.
    """
    S_actual = generar_solucion_inicial()
    costo_actual = calcular_costo_solucion(S_actual)
    
    S_mejor = [ruta[:] for ruta in S_actual]
    costo_mejor = costo_actual
    
    T = T_inicial
    costo_inicial_global = costo_actual 

    print("Iniciando Recocido Simulado...")
    print(f"Costo inicial: {costo_actual:.2f}")

    while T > T_final:
        for _ in range(max_iteraciones_T):
            S_vecina = generar_vecina(S_actual)
            costo_vecina = calcular_costo_solucion(S_vecina)
            
            delta_E = costo_vecina - costo_actual
            
            if delta_E < 0:
                # Aceptar solución mejor
                S_actual = S_vecina
                costo_actual = costo_vecina
                
                if costo_actual < costo_mejor:
                    S_mejor = [ruta[:] for ruta in S_actual]
                    costo_mejor = costo_actual
            else:
                # Aceptar solución peor (criterio de Metropolis)
                try:
                    probabilidad_aceptacion = math.exp(-delta_E / T)
                except OverflowError:
                    probabilidad_aceptacion = 0.0

                if random.random() < probabilidad_aceptacion:
                    S_actual = S_vecina
                    costo_actual = costo_vecina
        
        # Aplicar esquema de enfriamiento (Geométrico)
        T *= alpha
        
    print("Recocido Simulado finalizado.")
    return S_mejor, costo_mejor, costo_inicial_global

## Paso 4: Ejecución y Resultados

In [13]:
# Parámetros del Recocido Simulado 
T_INICIAL = 1000.0
ALPHA = 0.995
T_FINAL = 1.0
MAX_ITERACIONES_T = 100

# Ejecutar el Recocido Simulado
solucion_final, costo_final, costo_inicial = recocido_simulado(T_INICIAL, T_FINAL, ALPHA, MAX_ITERACIONES_T)

# Preparación de Resultados para el Reporte
resultados = []
for i, ruta in enumerate(solucion_final):
    if len(ruta) > 2:
        costo_ruta = calcular_costo_solucion([ruta])
        
        # Obtener nombres de los nodos
        nodos_ruta = [df_nodos.iloc[nodo]['Nombre'] for nodo in ruta]
        
        # Calcular distancia total de la ruta (para reporte de eficiencia en km)
        distancia_total_ruta = 0.0
        matriz_distancias = df_distancias[columnas_datos].values
        for k in range(len(ruta) - 1):
            nodo_inicio = ruta[k]
            nodo_fin = ruta[k+1]
            distancia_total_ruta += matriz_distancias[nodo_inicio, nodo_fin]

        resultados.append({
            'CD_Asignado': df_nodos.iloc[ruta[0]]['Nombre'],
            'Costo_Total_Ruta': costo_ruta,
            'Distancia_Total_Km': distancia_total_ruta,
            'Ruta_Secuencia': ' -> '.join(nodos_ruta)
        })

df_resultados = pd.DataFrame(resultados)
nombre_archivo_salida = "solucion_optima_mdvrp_recocido_simulado.csv"
df_resultados.to_csv(nombre_archivo_salida, index=False)

# Imprimir Resumen
print("\n" + "="*50)
print("           RESUMEN DE EJECUCIÓN DEL MDVRP")
print("="*50)
print(f"Costo de la solución inicial (Generada Aleatoriamente): {costo_inicial:.2f}")
print(f"Costo de la solución óptima encontrada (Recocido Simulado): {costo_final:.2f}")
print(f"Mejora: {((costo_inicial - costo_final) / costo_inicial) * 100:.2f}%")

print(f"\nDetalle de Rutas y Costos (Guardado en {nombre_archivo_salida}):")
print(df_resultados[['CD_Asignado', 'Costo_Total_Ruta', 'Distancia_Total_Km']].head())

Iniciando Recocido Simulado...
Costo inicial: 171.78
Recocido Simulado finalizado.

           RESUMEN DE EJECUCIÓN DEL MDVRP
Costo de la solución inicial (Generada Aleatoriamente): 171.78
Costo de la solución óptima encontrada (Recocido Simulado): 91.57
Mejora: 46.70%

Detalle de Rutas y Costos (Guardado en solucion_optima_mdvrp_recocido_simulado.csv):
                 CD_Asignado  Costo_Total_Ruta  Distancia_Total_Km
0   Centro de Distribución 4         47.935802          319.572014
1   Centro de Distribución 6          0.991543            6.610288
2   Centro de Distribución 8         18.796424          125.309490
3   Centro de Distribución 9          6.011340           40.075601
4  Centro de Distribución 10         17.830125          118.867497
